In [3]:
import cv2
import numpy as np
from ultralytics import YOLO
import pygame
import threading

# --------------------------
# Initialize Pygame mixer
# --------------------------
pygame.mixer.init()
alert_sound = r"D:\Sainath_Backup\Desktop\UCHIT TECHNOLOGIES\VMS\backend1\Person detection\alert.mp3"

def play_alert():
    if not pygame.mixer.music.get_busy():  # prevent overlapping sounds
        pygame.mixer.music.load(alert_sound)
        pygame.mixer.music.play()

# --------------------------
# Load YOLOv8 custom model
# --------------------------
# First make sure you have created person_intrusion.pt with:
# model = YOLO("yolov8n.pt"); model.save("person_intrusion.pt")
model = YOLO("person_intrusion.pt")  # your renamed model

# --------------------------
# Video path
# --------------------------
video_path = r"D:\Sainath_Backup\Desktop\UCHIT TECHNOLOGIES\VMS\backend1\Person detection\thief.mp4"
cap = cv2.VideoCapture(video_path)

# --------------------------
# Dynamic Zone Drawing
# --------------------------
zone_points = []
drawing = False

def draw_zone(event, x, y, flags, param):
    global zone_points, drawing
    if event == cv2.EVENT_LBUTTONDOWN:
        zone_points.append((x, y))
    elif event == cv2.EVENT_RBUTTONDOWN:
        drawing = False

# Grab first frame to draw zone
ret, first_frame = cap.read()
if not ret:
    print("Failed to read video")
    exit()

clone = first_frame.copy()
cv2.namedWindow("Draw Zone - Left click to select points, Right click to finish")
cv2.setMouseCallback("Draw Zone - Left click to select points, Right click to finish", draw_zone)
drawing = True

while drawing:
    temp = clone.copy()
    if len(zone_points) > 1:
        cv2.polylines(temp, [np.array(zone_points)], isClosed=True, color=(0,0,255), thickness=2)
    cv2.imshow("Draw Zone - Left click to select points, Right click to finish", temp)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC to cancel
        drawing = False

cv2.destroyWindow("Draw Zone - Left click to select points, Right click to finish")

# --------------------------
# Function to check if point is inside polygon
# --------------------------
def point_in_polygon(point, polygon):
    return cv2.pointPolygonTest(np.array(polygon, np.int32), point, False) >= 0

# --------------------------
# Process video frames
# --------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Draw the polygon zone
    if len(zone_points) > 1:
        cv2.polylines(frame, [np.array(zone_points)], isClosed=True, color=(0,0,255), thickness=2)

    results = model(frame)[0]
    intruder_detected = False

    for box, cls in zip(results.boxes.xyxy, results.boxes.cls):
        if int(cls) == 0:  # person
            x1, y1, x2, y2 = map(int, box)
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2

            if point_in_polygon((cx, cy), zone_points):
                intruder_detected = True
                cv2.putText(frame, "INTRUDER DETECTED!", (x1, y1-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0,0,255), 2)
                cv2.circle(frame, (cx, cy), 5, (0,0,255), -1)
            else:
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
                cv2.circle(frame, (cx, cy), 5, (0,255,0), -1)

    if intruder_detected:
        threading.Thread(target=play_alert, daemon=True).start()

    cv2.imshow("Intrusion Detection", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC to exit
        break

cap.release()
cv2.destroyAllWindows()
pygame.mixer.quit()



0: 352x640 1 person, 38.2ms
Speed: 6.6ms preprocess, 38.2ms inference, 6.5ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 person, 56.1ms
Speed: 14.3ms preprocess, 56.1ms inference, 8.2ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 person, 38.8ms
Speed: 6.9ms preprocess, 38.8ms inference, 5.8ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 person, 35.4ms
Speed: 7.0ms preprocess, 35.4ms inference, 3.9ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 person, 43.5ms
Speed: 7.6ms preprocess, 43.5ms inference, 10.1ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 person, 76.0ms
Speed: 11.0ms preprocess, 76.0ms inference, 7.9ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 person, 35.8ms
Speed: 8.3ms preprocess, 35.8ms inference, 6.0ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 person, 31.2ms
Speed: 5.1ms preprocess, 31.2ms inference, 3.5ms postprocess per image at shape (1, 3,

In [2]:
from ultralytics import YOLO

# Load the pretrained YOLOv8 model
model = YOLO("yolov8n.pt")

# Save it with a new name (no retraining)
model.save("person_intrusion.pt")


In [3]:
from ultralytics import YOLO

model = YOLO("D:/Sainath_Backup/Desktop/UCHIT TECHNOLOGIES/final_submission/backend/vest_model.pt")
print(model.names)

{0: 'no-vest', 1: 'vest'}
